In [760]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnableLambda
import base64
from PIL import Image
import io
import os
import pandas as pd
from tabulate import tabulate

In [761]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [762]:
def prompt_func(data):
    text = data["text"]
    image = data["image"]
    #system_message = SystemMessage(content=data["system_message_text"])
    content_parts = []

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }
    text_part = {"type": "text", "text": text}
    content_parts.append(image_part)
    content_parts.append(text_part)

    human_message = HumanMessage(content=content_parts)

    #return [system_message, human_message]
    return [human_message]

In [763]:
def get_image_paths(directory):
    allowed_extensions = {".jpg", ".jpeg", ".png"}
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.splitext(filename)[1].lower() in allowed_extensions]
    return image_paths

In [764]:
def create_df(image_paths):
    image_list = []
    for path in image_paths:
        image = Image.open(path)
        image_name = os.path.basename(path)
        exifdata = image._getexif()
        date_time, localisation = None, None
        if exifdata:
            for tag_id, value in exifdata.items():
                tag = Image.ExifTags.TAGS.get(tag_id, tag_id)
                if tag == "DateTime":
                    date_time = value
                elif tag == "GPSInfo":
                    localisation = value

            image_list.append((image_name, path, date_time, localisation))

        else:
            print("Aucune donnée EXIF trouvée.")

    df = pd.DataFrame(image_list, columns=["image_name", "path", "date_time", "localisation"])
    df["keywords"] = ""
    df["categories"] = ""

    return df

In [765]:
def add_keywords_to_df(image_data, keywords_output):
    if keywords_output:
        # Mise à jour uniquement pour les images présentes dans keywords_output
        image_data["keywords"] = image_data.apply(
            lambda row: keywords_output[row["image_name"]]
            if row["image_name"] in keywords_output else row["keywords"], axis=1
        )
    else:
        print("Aucun mot clé fourni ! ")
    return image_data

In [766]:
def get_missing_values(dictionnary):
    missing_values = {}
    for key, value in dictionnary.items():
        if value is None or value == "" or value == "nan" or value == "None":
            missing_values.update({key: value})

    return missing_values

In [767]:
def get_missing_path(paths, dictionnary):
    missing_paths = []
    for path in paths:
        image_name = os.path.split(path)[-1]
        #print(f"image_name : {image_name}")
        if image_name in dictionnary.keys() or path in dictionnary.keys():
            missing_paths.append(path)

    return missing_paths

In [768]:
def get_missing_values_path(paths, dictionnary):
    missing_values = get_missing_values(dictionnary)
    missing_paths = get_missing_path(paths, missing_values)
    print(f"Images détectées avec valeurs manquantes : {missing_values.keys()}")
    print(f"Chemins renvoyés pour traitement : {missing_paths}")
    return missing_paths

In [769]:
def checking_all_keywords(df):
    path_images_empty = []
    none_possibilities = [None, "", [], "None", ["None"]]
    for row in df.itertuples():
        keywords = row.keywords
        if isinstance(keywords, float) and pd.isna(keywords):
            path_images_empty.append(row.path)

        elif keywords in none_possibilities:
            path_images_empty.append(row.path)

    return path_images_empty

In [770]:
def keywords_call(df, image_paths, keywords_chain):
    for i in range(0, len(image_paths)):
        print(f"Image {i} : {image_paths[i]}")
        image_b64 = encode_image(image_paths[i])
        image_name = os.path.basename(image_paths[i])

        keywords_output = keywords_chain.invoke({"text":f"""Décris moi l'image avec 5 mots-clés. Les mots-clés doivent en priorité inclure des actions, des objets et un lieu si identifiables. Les mots-clés doivent être en français, et peuvent être des mots composés
        Retourne le résultat au format JSON suivant: {{ {image_name} : [mot-clé1, mot-clé2, mot-clé3, mot-clé4, mot-clé5] }}""", "image": image_b64})

        print(f"Keywords : {keywords_output}")

        df = add_keywords_to_df(df, keywords_output)

    return df

In [771]:
def pipeline_keywords(image_paths):
    image_data = create_df(image_paths)
    new_image_paths = image_paths

    llm = ChatOllama(model="gemma3")

    prompt_chain = RunnableLambda(prompt_func)
    keyword_chain = prompt_chain | llm | JsonOutputParser()

    all_keywords = False
    only_once = False

    while not all_keywords :
        print("Entree dans le while")
        image_data = keywords_call(image_data, new_image_paths, keyword_chain)

        if only_once:
            new_row = pd.DataFrame([{"image_name" : "IMG_20241228_132157.jpg","path": "photos_victor/IMG_20241228_132157.jpg"}])
            image_data = pd.concat([image_data, new_row], ignore_index=True)
            only_once = False

        new_image_paths = checking_all_keywords(image_data)
        print(f"Images à traiter après le premier passage : {new_image_paths}")

        if not new_image_paths:
            all_keywords = True

    return image_data

In [772]:
image_paths = get_image_paths("test_data")
image_data = pipeline_keywords(image_paths)

Entree dans le while
Image 0 : test_data\20240822_142034.jpg
Keywords : {'20240822_142034.jpg': ['Bâtiment', 'Rue', 'Perspective', 'Ville', 'Voiture']}
Image 1 : test_data\20240822_142036.jpg
Keywords : {'20240822_142036.jpg': ['Bâtiment', 'Fenêtres', 'Rue', 'Bricks', 'Éclair']}
Image 2 : test_data\20240822_142048.jpg
Keywords : {'20240822_142048.jpg': ['Architecture', 'Ville', 'Voiture', 'Pluie', 'Reflets']}
Image 3 : test_data\20240825_121144.jpg
Keywords : {'20240825_121144.jpg': ['Pot', 'Beurrefaits', 'Kraft', 'Plan de travail', 'Texture']}
Image 4 : test_data\20240826_184159.jpg
Keywords : {'20240826_184159.jpg': ['Chemin', 'Forêt', 'Ombres', 'Sentier', 'Arbres']}
Image 5 : test_data\20240827_084810.jpg
Keywords : {'20240827_084810.jpg': ['Bâtiments', 'Gazon', 'Pavillon', 'Ciel', 'Terrasse']}
Image 6 : test_data\20240827_084914.jpg
Keywords : {'20240827_084914.jpg': ['Tentes', 'Allée', 'Événement', 'Personnes', 'Éclairage']}
Image 7 : test_data\20240827_092136.jpg
Keywords : {'202

In [773]:
print(tabulate(image_data, headers="keys", tablefmt="psql"))

+----+-------------------------+---------------------------------------+---------------------+---------------------------------------------------------------------+-------------------------------------------------------------------------+--------------+
|    | image_name              | path                                  | date_time           | localisation                                                        | keywords                                                                | categories   |
|----+-------------------------+---------------------------------------+---------------------+---------------------------------------------------------------------+-------------------------------------------------------------------------+--------------|
|  0 | 20240822_142034.jpg     | test_data\20240822_142034.jpg         | 2024:08:22 14:20:34 |                                                                     | ['Bâtiment', 'Rue', 'Perspective', 'Ville', 'Voiture']                  |